In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression, fetch_california_housing

In [ ]:
## Create Data
# housing = fetch_california_housing()
# print(housing.data.shape, housing.target.shape)

In [ ]:
def get_metrics(ytrue: np.array, ypred: np.array):
    error = y-ypred
    mae = np.sum(abs(error))/y.shape[0]
    mse = np.sum((error)**2)/y.shape[0]
    rmse = mse**0.5
    return {"MSE": mse, "MAE": mae, "RMSE": rmse}

In [ ]:
# Linear Regression Formula
# Normal Equation: theta = (X^T * X)^-1 * X^T * y => here X is the input ndarray (matrix) and y is the target

## Normal Form

In [ ]:
class LinReg_Norm:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.theta = None
        
    def fit(self):
        # normal equation: theta = (X^T * X)^-1 * X^T * y
        
        # We need to append a row on 1's as an itercept to the input X array.
        X_1 = np.c_[np.ones((self.X.shape[0], 1)), self.X]
        X_1T = X_1.T
        self.theta = np.linalg.inv(X_1T.dot(X_1)).dot(X_1T).dot(y)

    def predict(self, X_test):
        # We need to append a row on 1's as an itercept to the input X_test array.
        X_test1 = np.c_[np.ones((X_test.shape[0], 1)), X_test]
        y_pred = X_test1.dot(self.theta)
        return y_pred
        
    def fit_predict(self):
        self.fit()
        return self.predict(self.X)
        

In [ ]:
X, y = make_regression(n_samples=100, n_features=10, noise=5, random_state=42)

lr_norm = LinReg_Norm(X, y)

In [ ]:
lr_norm.fit()
ypred = lr_norm.predict(X)

In [ ]:
get_metrics(y, ypred)

In [ ]:
plt.plot(ypred, marker='x', color='red', linestyle='None', label='y_pred')
plt.plot(y, marker='.', color='blue', linestyle='None', label='y_true')
plt.legend()
plt.show()

## Gradient Descent

$$ \text{MSE Loss Function: } J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right)^2 $$
    

$$ \text{Loss Function Partial Derivative: } \frac{\partial}{\partial \theta_j} J(\theta) = \frac{1}{m} \sum_{i=1}^{m} \left( h_{\theta}(x^{(i)}) - y^{(i)} \right) x_j^{(i)} $$

$$ \text{The hypothesis function can be written in matrix form as } h_{\theta}(X)=X{\theta}$$

$$ \text{Vectorized Gradient Calculation: } \nabla_{\theta} J(\theta) = \frac{1}{m} X^T (X\theta - y) $$

$$ \text{Gradient} = \frac{1}{m} X^T (X\theta - y) $$

$$ \text{Gradient Update: } \theta_j := \theta_j - \alpha \frac{\partial}{\partial \theta_j} J(\theta) $$


In [ ]:
def linreg_gradient_descent(X, y, learning_rate, num_iterations, batch_size):
    num_samples, num_features = X.shape
    theta = np.zeros((num_features, 1))
    cost_history = []
    
    for i in range(num_iterations):
        total_cost = 0.0
        
        indices = np.arange(num_samples)
        np.random.shuffle(indices)
        X_shuffled = X[indices]
        y_shuffled = y[indices]
        
        for j in range(0,num_samples,batch_size):
            x_batch = X_shuffled[j:batch_size+j]
            y_batch = y_shuffled[j:batch_size+j]
            
            # Compute prediction and error for the single batch
            prediction = np.dot(x_batch, theta)
            error = prediction - y_batch

            # Update parameters using the gradient for the single batch
            gradient = (1 / batch_size) * np.dot(x_batch.T, error)
            theta -= learning_rate * gradient
            
            cost_batch = (1 / (2 * batch_size)) * np.sum(np.square(error))

            total_cost += cost_batch
        
        avg_cost = total_cost / (num_samples/batch_size)
        cost_history.append(avg_cost)
    
    return theta, cost_history

In [ ]:
class LinReg_GradDescent:
    def __init__(self, X, y):
        self.X = np.c_[np.ones((X.shape[0], 1)), X]
        self.y = y.reshape(-1, 1)
        self.theta = None
        
    def fit(self, learning_rate, num_iterations, batch_size):
        num_samples, num_features = self.X.shape
        theta = np.zeros((num_features, 1))
        theta
        cost_history = []

        for i in range(num_iterations):
            total_cost = 0.0

            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            X_shuffled = self.X[indices]
            y_shuffled = self.y[indices]

            for j in range(0,num_samples,batch_size):
                x_batch = X_shuffled[j:batch_size+j]
                y_batch = y_shuffled[j:batch_size+j]

                prediction = np.dot(x_batch, theta)
                error = prediction - y_batch

                gradient = (1 / batch_size) * np.dot(x_batch.T, error)
                theta -= learning_rate * gradient


                cost_batch = (1 / (2 * batch_size)) * np.sum(np.square(error))

                total_cost += cost_batch

            avg_cost = total_cost / (num_samples/batch_size)
            cost_history.append(avg_cost)
        
        self.theta = theta
        return cost_history

    def predict(self, X_test):
        # We need to append a row on 1's as an itercept to the input X_test array.
        X_with_intercept = np.c_[np.ones((X_test.shape[0], 1)), X_test]
        y_pred = np.dot(X_test_with_intercept, self.theta)

        return y_pred

In [ ]:
lr_grad = LinReg_GradDescent(X, y)

In [ ]:
cost_history_1 = lr_grad.fit(learning_rate=0.01, num_iterations=1000, batch_size=1)
cost_history_25 = lr_grad.fit(learning_rate=0.01, num_iterations=1000, batch_size=25)
cost_history_100 = lr_grad.fit(learning_rate=0.01, num_iterations=1000, batch_size=100)

In [ ]:
plt.plot(cost_history_g, color='red', linestyle='-', label='y_pred')
plt.plot(cost_history_1, color='blue', linestyle='-', label='y_pred')
plt.plot(cost_history_25, color='green', linestyle='-', label='y_pred')
plt.plot(cost_history_100, color='yellow', linestyle='-.', label='y_pred')

## Assumptions in Linear Models

In [ ]:
# 1. Linearity - Check scatter plots of features vs. target
for i in range(X.shape[1]):
    plt.scatter(X[:,i], y)
    plt.xlabel(f'Feature_{i}')
    plt.ylabel('Target')
    plt.title(f'Feature_{i} vs. Target')
    plt.show()

In [ ]:
# 2. Independence - Check residuals vs. predicted values
y_pred = lr_grad.predict(X).reshape(-1)
residuals = y - y_pred
plt.scatter(y_pred, residuals)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs. Predicted Values')
plt.axhline(y=0, color='red', linestyle='--')  # Add a horizontal line at y=0 for reference
plt.show()

In [ ]:
# 3. Homoscedasticity - Check residuals vs. predicted values for constant spread
plt.scatter(y_pred, np.abs(residuals))
plt.xlabel('Predicted Values')
plt.ylabel('Absolute Residuals')
plt.title('Absolute Residuals vs. Predicted Values')
plt.axhline(y=0, color='red', linestyle='--')
plt.show()

In [ ]:
# 4. Normality of Residuals - Check histogram and Q-Q plot of residuals
plt.hist(residuals, bins=20)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()

In [ ]:
import scipy.stats as stats
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.show()

In [ ]:
# 5. No Multicollinearity - Check correlation matrix and VIF values
# correlation_matrix = X.corr()
correlation_matrix = pd.DataFrame(X).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X, i) for i in range(X.shape[1])]
print('VIF values:')
print(vif)